<a href="https://colab.research.google.com/github/jeangreg89/dio-bd-relacional/blob/main/Projeto_Jean_Greguere.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [41]:
# Extração dos dados
import pandas as pd
import json
import sqlite3

df= pd.read_excel ("BP.xlsx")
print(df.head(),"\n")

                    Conta  Empresa
0  Aplicações Financeiras     2600
1                  Bancos      800
2                   Caixa    24380
3          Capital Social    10000
4          Contas a pagar      780 



In [40]:
# Transformação Dos dados
import pandas as pd

df = pd.read_excel("BP.xlsx") # Recarregue o DataFrame para garantir dados atualizados para as transformações.

# Função para classificar as contas
def classificar_conta(nome_conta):
  nome_conta = nome_conta.strip().lower()

  # Lista de contas do Ativo Circulante (AC)
  ativo_circulante = [
      "caixa",
      "bancos",
      "aplicações financeiras",
      "cliente",
      "estoques"
  ]


  # Lista de contas do Ativo Não Circulante (ANC)

  ativo_nao_circulante  = [
      "imóveis",
      "máquinas",
      "veículos",
  ]



  # Lista de contas do Passivo Circulante (PC)
  passivo_circulante  = [
      "contas a pagar",
      "empréstimos a pagar",
      "fornecedores",
      "impostos a pagar",
  ]


  # Lista de contas do Passivo Não Circulante (PNC)

  passivo_nao_circulante  = [
      "financiamentos (longo prazo)"
  ]


  # Lista de contas do Patrimônio Líquido (PL)

  patrimonio_liquido = [
      "capital social",
       "lucros (prejuízos) acumulados"
  ]

  if nome_conta in [x.lower().strip() for x in ativo_circulante]:
    return "Ativo Circulante (AC)"
  elif nome_conta in [x.lower().strip() for x in ativo_nao_circulante]:
    return "Ativo Não Circulante (ANC)"
  elif nome_conta in [x.lower().strip() for x in passivo_circulante]:
    return "Passivo Circulante (PC)"
  elif nome_conta in [x.lower().strip() for x in passivo_nao_circulante]: # Adicionada esta categoria
    return "Passivo Não Circulante (PNC)"
  elif nome_conta in [x.lower().strip() for x in patrimonio_liquido]:
    return "Patrimônio Líquido (PL)"
  else:
    return "Não classificado"

# Aplicar a classificação
df["Grupo contábil"] = df["Conta"].apply(classificar_conta)

# Converter valores numéricos (retirar vírgulas, R$, .....)
# Considerando o formato numérico Brasileiro: o separador de milhare é '.', o separador decimal é ','
df["Empresa"] = (
    df["Empresa"].astype(str)
    .str.replace("R$", "", regex=True) # Remove currency symbol R$
    .str.replace(".", "", regex=False) # Remove thousands separator '.'
    .str.replace(",", ".", regex=False) # Replace decimal separator ',' with '.'
    .astype(float)
)

#Somar por grupo contábil

saldos = df.groupby("Grupo contábil")["Empresa"].sum()

#Calcular totais e verificar equação contábil

ativo_total = saldos.get("Ativo Circulante (AC)", 0) + saldos.get("Ativo Não Circulante (ANC)", 0)
passivo_total = saldos.get("Passivo Circulante (PC)", 0) + saldos.get("Passivo Não Circulante (PNC)", 0)
pl_total = saldos.get("Patrimônio Líquido (PL)", 0)

#Mostrar resultados

print("=== SALDOS POR GRUPO ===")
print(saldos)
print("\n=== EQUAÇÃO CONTÁBIL ===")
print(f"Ativo Total: {ativo_total:,.2f}")
print(f"Passivo + PL: {passivo_total + pl_total:,.2f}")

if abs(ativo_total - (passivo_total + pl_total)) < 0.01:
  print("A equação está correta.")
else:
  print("A equação não está correta.")

=== SALDOS POR GRUPO ===
Grupo contábil
Ativo Circulante (AC)            53680.0
Ativo Não Circulante (ANC)      113600.0
Passivo Circulante (PC)          56780.0
Passivo Não Circulante (PNC)    100000.0
Patrimônio Líquido (PL)          10500.0
Name: Empresa, dtype: float64

=== EQUAÇÃO CONTÁBIL ===
Ativo Total: 167,280.00
Passivo + PL: 167,280.00
A equação está correta.
